<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Алексей, привет! Спасибо, что прислал работу во время :) 

Приношу свои извинения от лица компании, что задержались с проверкой твоей работы :( Постараемся больше так не делать.

Меня зовут Евгений, и я буду проверять твой проект) 
Ты проделал большую работу над проектом, однако в любом деле всегда можно можно что-то улучшить. Моя основная цель — не показать, что где—то совершены какие—либо ошибки, а поделиться своим опытом, который поможет тебе в дальнейших работах. Далее в файле ты сможешь увидить мои комментарии, постарайся учесть их при выполнении следующих заданий.    

Мои комментарии обозначены пометкой **Комментарий ревьюера**. Далее в файле ты сможешь найти их в похожих ячейках (если фон комментария <font color=green>зелёный</font> - всё сделано правильно; <font color=orange>оранжевый</font> - есть замечания, но не критично; <font color=red>красный</font> - нужно переделать). Не удаляй эти комментарии и постарайся учесть их в ходе выполнения данного проекта. 

</div>

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-warning">
<h2> Комментарий ревьюера</h2>

Рекомендуется делать краткое вступление в работу и описание того, что планируешь сделать (если план у тебя уже сформирован), а также информацию о входных данных: какие столбцы есть в таблице, их названия и какую информацию они несут. Так работа станет выглядеть презентабельнее.

</div>

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
client_solvency=pd.read_csv('/datasets/data.csv')
#print(client_solvency.head(10))
#print(client_solvency.tail(10))
#Посмотрим информацию об типах данных и их количестве
print(client_solvency.info())
#Посмотрим общую информацию о количественных данных
print(client_solvency.describe())
print(client_solvency['children'].value_counts())
#И отдельно по столбцам с качественными данными
print(client_solvency['education'].value_counts())
print(client_solvency['family_status'].value_counts())
print(client_solvency['gender'].value_counts())
print(client_solvency['income_type'].value_counts())
print(client_solvency['purpose'].value_counts().head(10))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
           children  days_employed     dob_years  education_id  \
count  21525.000000   19351.000000  21525.000000  21525.000000   
mean       0.538908   63046.497661     43.293380      0.817236   
std        1.381587  140827.311974     12.574584      0.548138   
min       -1.000000  -18388.949901      0.000000      0.000

### Вывод

При просмотре данных обнаружено следующее:
1. 12 колонок с различной информацией о платёжеспособности клиентов, есть количественные данные (float64(2), int64(5)) и качественные (object(5))
2. В колонках days_employed(общий трудовой стаж в днях) и total_income(ежемесячный доход) обнаружены пропуски.
3. В колонках days_employed(общий трудовой стаж в днях) и total_income(ежемесячный доход) тип данных float64, хотя по своей сути можно оперировать целыми числами для этих колонок (int64).
4. В колонке education есть записи, которые имееют однаковое значение (пример - "среднее, Среднее, СРЕДНЕЕ"), но написаны в разном регистре. 
5. В колонке purpose очень много похожих причин для взятия кредита, написанных по-разному (например "свадьба, на проведение свадьбы, сыграть свадьбу"), которые нужно будет объеденить.
6. Обнаружены аномалии в количественных данных:
- days_employed - большинство значений отрицательные, есть аномально высокие значения не сопоставимые с средней продолжительностью человеческой жизни.
- family_status - помимо стандартных мужчина (М) и женщина (F), обнаружено 1 непонятное значение XNA. Но поскольку это только одно значение из 21525 - им можно принебречь
- children - есть значения количества детей с отрицательным значением ("-1" - 47 строк) - и аномально высоким ("20" - 76 строк).
- dob_years - есть значения возраста, равное "0", новорожденные не могут брать кредиты, а судя по другим строкам, где встречается 0, ещё работать, получать пенсию ит.д. - это ошибка, нулевые значения нужно убрать.


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Хорошо. Сырые данные качественно изучили, исходя из этого мы можем дальше выполнять свой анализ.
Давай теперь постараемся их обработать.

</div>

### Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
#поменяем отрицательные значения стажа на положительные
client_solvency['days_employed']=abs(client_solvency['days_employed'])
#посчитаем средние значения дохода и трудового стажа
#days_employed_avg=client_solvency['days_employed'].mean()
days_employed_avg=client_solvency['days_employed'].median()
print(days_employed_avg)
print()
#total_income_avg=client_solvency['total_income'].mean()
total_income_avg=client_solvency['total_income'].median()
print(total_income_avg)
print()
#Заменим пропуски в данных по стажу и доходу на медиану 
client_solvency['days_employed']= client_solvency['days_employed'].fillna(value=days_employed_avg)
client_solvency['total_income']= client_solvency['total_income'].fillna(value=total_income_avg)

print(client_solvency.info())


#высчитаем средний возраст
dob_years_avg=client_solvency['dob_years'].mean()
#уберём пропуски (нулевые значения) в данных о возрасте
client_solvency['dob_years']= client_solvency['dob_years'].fillna(value=total_income_avg)
#проверим количество нулевых значений 
print(client_solvency['dob_years'].isnull().sum())
#нулевые значения убраны, ура

#теперь поработаем с ошибками в колонке с детьми, для начала уберём минусовые значения
client_solvency['children']=abs(client_solvency['children'])

#затем уберём аномально высокое количество детей
client_solvency['children']= client_solvency['children'].replace(20, 0)
print(client_solvency['children'].value_counts())

    

2194.220566878695

145017.93753253992

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None
0
0    14225
1     4865
2     2055
3      330
4       41
5        9
Name: children, dtype: int64


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Хорошо, данные обновили. Если посмотерть на уровень дохода более детально, используя например профессию, уровень образования — сильно ли будут отличаться данные от общей медианы в выборке ? 
    
Какая доля клиентов с некорректными данными по возрасту детей, можно было бы удалить эти данные ?
    
</div>

### Вывод

1. Обнаружены пропущенные значения в трудовом стаже и доходе клиента (days_employed, total_income).  
2. Пропуска в строках по колонкам 'days_employed' и 'total_income' совпадают, что может означать, что эти данные были не заполнены при подаче анкеты либо это ошибка выгрузки. Все остальные колонки заполнены по этим строкам. 
3. Для заполнения пропусков, посчитал сначала средние значения, затем медианы, остановился на последних, потому что разница очень большая по значениям между средним и медианой.
Замечания: Непонятны значения стажа 'days_employed', подавляющее большинство значений - минусовые. А некоторые плюсовые - аномально высокие. Поэтому отрицательные значения были переведены в положительные, и взята медиана.

### Замена типа данных

In [3]:
#заменим данные с дробными числами на целые
client_solvency['days_employed ']=client_solvency['days_employed'].astype('int')
client_solvency['total_income']=client_solvency['total_income'].astype('int')
print(client_solvency.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
days_employed       21525 non-null int64
dtypes: float64(1), int64(7), object(5)
memory usage: 2.1+ MB
None


1. Тип данных в столбцах [days_employed] и ['total_income'] заменён с float64 (дробные числа) на int64 (целые числа).
Для стажа работы, который принято считать как минимум в днях (а то и месяцах,годах) значения после запятой не учитываются.
Для общего дохода, значения после запятой - это копейки, которыми можно принебречь.

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Какая еще может быть причина замены типа данных? 

</div>

### Обработка дубликатов

In [4]:


#Уберём одинаковые значения образования, заполненные в разном регистре:
client_solvency['education'] = client_solvency['education'].str.lower()
print(client_solvency['education'].unique())
#Уберём одинаковые значения образования, заполненные в разном регистре:
client_solvency['family_status'] = client_solvency['family_status'].str.lower()
print(client_solvency['family_status'].unique())
#посмотрим уникальные столбцы по не количественным столбцам
print(client_solvency['income_type'].unique())
#print(client_solvency['purpose'].unique())



client_solvency=client_solvency.drop_duplicates().reset_index(drop = True)
print(client_solvency[client_solvency.duplicated() == True])

print('Дубликатов в таблице:',client_solvency.duplicated().sum())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
Empty DataFrame
Columns: [children, days_employed, dob_years, education, education_id, family_status, family_status_id, gender, income_type, debt, total_income, purpose, days_employed ]
Index: []
Дубликатов в таблице: 0


### Вывод

поясните, как выбирали метод для поиска и удаления дубликатов в данных; 
1. При первичном поиске дубликатов методом .duplicated(), количество дубликатов было равно 54. Поскольку в предобработке данных пропуски были заменены на медиану, столбцы с количественными значениями ['days_employed '] и ['total_income']  детально не проверялись, т.к. уникальных значений слишком много. Методом .unique()) были просмотрены столбцы с типом данных object и была замечена особенность написания образования ['education'], одни и те же значения были написаны в разном регистре.  Методом .str.lower() были переведены значение регистра в стобце eductaion  в нижний регистр, и после этого количество дубликатов увеличилось до 71. Поскольку удалять дубликаты по данным только одного из столбцов некорректно, дубликаты удалялись по полностью повторяющимся строкам методом drop_duplicates()
приведите возможные причины появления дубликатов;
2. Поскольку в каждом из дубликатов встречается значение дохода и стажа, которое было заменено на медиану(изначальный пропуск), то возможные причины - это некорректная подача анкеты, а затем внесение новой анкеты без удаления предыдущей.

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.

</div>

### Лемматизация

In [5]:
#Выделим леммы столбца purpose
from pymystem3 import Mystem
m = Mystem()
def lemmatize (row):
    purpose = row['purpose']
    lemmas = m.lemmatize(purpose)
    return lemmas


#добавим дополнительный столбец с леммами
client_solvency['purpose_lemm']=client_solvency.apply(lemmatize, axis=1)

#разделим леммы по категориям c учётом основных причин - автомобиль, свадьба, недвижимость, образование, коммерческая недвижимость
def purpose_reason(row):
    purpose_reason_word = row['purpose_lemm']
    if 'автомобиль' in purpose_reason_word:
        return 'Покупка авто'
    if 'свадьба' in purpose_reason_word:
        return 'Деньги на свадьбу'
    if 'образование' in purpose_reason_word:
        return 'Кредит на образование'
    if 'коммерческий' in purpose_reason_word:
        return 'Коммерческая недвижимость'
    return 'Покупка недвижимости'

client_solvency['purpose_lemm_category']=client_solvency.apply(purpose_reason, axis=1)
print(client_solvency['purpose_lemm'].value_counts())
print(client_solvency['purpose_lemm_category'].value_counts())

#для удобства дальнейшей обработки добавим числовые значения категорий по целям кредита

def purpose_index(row):
    purpose_index_phrase = row['purpose_lemm_category']
    if 'Покупка авто' in purpose_index_phrase:
        return '1'
    if 'Деньги на свадьбу' in purpose_index_phrase:
        return '2'
    if 'Кредит на образование' in purpose_index_phrase:
        return '3'
    if 'Коммерческая недвижимость' in purpose_index_phrase:
        return '4'
    return '5'

client_solvency['purpose_index']=client_solvency.apply(purpose_index, axis=1)
print(client_solvency['purpose_index'].value_counts())






[автомобиль, \n]                                          972
[свадьба, \n]                                             791
[на,  , проведение,  , свадьба, \n]                       768
[сыграть,  , свадьба, \n]                                 765
[операция,  , с,  , недвижимость, \n]                     675
[покупка,  , коммерческий,  , недвижимость, \n]           661
[операция,  , с,  , жилье, \n]                            652
[покупка,  , жилье,  , для,  , сдача, \n]                 651
[операция,  , с,  , коммерческий,  , недвижимость, \n]    650
[покупка,  , жилье, \n]                                   646
[жилье, \n]                                               646
[покупка,  , жилье,  , для,  , семья, \n]                 638
[строительство,  , собственный,  , недвижимость, \n]      635
[недвижимость, \n]                                        633
[операция,  , со,  , свой,  , недвижимость, \n]           627
[строительство,  , жилой,  , недвижимость, \n]            624
[покупка

### Вывод

выделите леммы в значениях столбца с целями получения кредита:
1. Леммы в столбце purpose выделены в отдельный столбец 'purpose_lemm'
опишите, как вы проводили лемматизацию целей кредита;
2. Данные были разделены на 5 категорий, поскольку в причинах есть разные значения недвижимости (недвижимость, коммерческая недвижимость, жильё), недвижимость разделена на коммерческую и обычную. Далее столбцы по остальным причинам (авто, свадьба, образование)

<div class="alert alert-warning">
<h2> Комментарий ревьюера</h2>

Верно. Рекомендую последовательно выполнять действия в работе, чтобы было видно что сделано и какие выводы и дальнейшие шаги мы предпринимаем в проекте. То есть, изначально определили леммы — сделали промежуточный вывод — написали функцию категоризации. 

Хоть функции и просты, напомню про отказоустойчивость кода с использованием связки «try—except», для создания будущих функций :)

</div>

### Категоризация данных

In [6]:

#разделим размер дохода по категориям низкий, средний, средний плюс, высокий, очень высокий, миллионер (6 типов)
def income_category(row):
     if row <= 60000:  
       return 'низкий доход'
     elif 60000 < row <= 120000:
       return 'средний доход'
     elif 120000 < row <= 180000:
       return 'доход выше среднего'
     elif 180000 < row <= 400000:
       return 'высокий доход'
     elif 400000 < row <= 1000000:
       return 'очень высокий доход'
    
     else:
       return 'рублёвые миллионеры'
client_solvency['income_categories'] = client_solvency['total_income'].apply(income_category)
print(client_solvency['income_categories'].value_counts())

#разделим dob_years по возрастным группам
def age_category(row):
     
     if row <= 30:  
       return 'Молодёжь'
     elif 30 < row <= 45:
       return 'Взрослые 30-45'
     elif 45 < row <= 60:
       return 'Взрослые 45-60'
     else:
       return 'Пожилые'
client_solvency['dob_years_categories'] = client_solvency['dob_years'].apply(age_category)
print(client_solvency['dob_years_categories'].value_counts())

#разделим children на категории
def children_count(row):
     
     if row == 0:  
       return 'Нет детей'
     elif row <= 1:
       return 'Один ребёнок'
     elif row <= 2:
       return 'Двое детей'
     else:
       return 'Многодетный'
client_solvency['children_count_categories'] = client_solvency['children'].apply(children_count)
print(client_solvency['children_count_categories'].value_counts())

#проверим категоризацию методом try-except


доход выше среднего    8187
средний доход          6039
высокий доход          5894
низкий доход            806
очень высокий доход     503
рублёвые миллионеры      25
Name: income_categories, dtype: int64
Взрослые 30-45    8486
Взрослые 45-60    7024
Молодёжь          3818
Пожилые           2126
Name: dob_years_categories, dtype: int64
Нет детей       14167
Один ребёнок     4855
Двое детей       2052
Многодетный       380
Name: children_count_categories, dtype: int64


<div class="alert alert-warning">
<h2> Комментарий ревьюера</h2>

Было бы интересно видеть твои размышления касаемо определения категорий по уровню доходов клиентов.
</div>

### Вывод

<div class="alert alert-warning">
<h2> Комментарий ревьюера</h2>

Было бы здорово убирать не используемые ячейки, то есть которые не несут в себе никакой информаци.

http://joxi.ru/1A5PY0oTGaMPV2

</div>

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
#создаем сводную таблицу
children_status_data=client_solvency.pivot_table(index = ['children_count_categories'], columns = 'debt', values = 'children', aggfunc='count')
 
#переименовываем колонки
new_columns = ['платежи в срок','просроченные платежи']
children_status_data.set_axis(new_columns, axis='columns',inplace = True)
 
#создаем новый столбец с долей и выводим таблицу на печать
children_status_data['доля просроченных платежей']=children_status_data['просроченные платежи']/(children_status_data['просроченные платежи']+children_status_data['платежи в срок'])
print(children_status_data)

                           платежи в срок  просроченные платежи  \
children_count_categories                                         
Двое детей                           1858                   194   
Многодетный                           349                    31   
Нет детей                           13096                  1071   
Один ребёнок                         4410                   445   

                           доля просроченных платежей  
children_count_categories                              
Двое детей                                   0.094542  
Многодетный                                  0.081579  
Нет детей                                    0.075598  
Один ребёнок                                 0.091658  


<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Свооодные таблицы — наше всё для быстрых вычислений :)
    
Верно.
    
</div>

<div class="alert alert-warning">
<h2> Комментарий ревьюера</h2>

Я бы рекомендовал не использовать атрибут **inplace**, так как в последующих обновлениях библотеки **pandas** он будет удален.
</div>

### Вывод

Исходя из полученных данных можно обозначить следующее:
1. Люди, у которых нет детей реже задерживают платежи
2. Больше всех задерживают платежи, люди, укоторых двое детей, чуть реже с одним ребёнком.
3. Однако люди, у которых 3 и более ребёнка реже делают просрочки по платежам, чем люди с одним или двумя детьми.

Как итог, зависимость между наличием детей и возвратом кредита в срок есть. 

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно, небольшая зависимость присутствует.
    
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
#создаем сводную таблицу
family_status_data=client_solvency.pivot_table(index = ['family_status'], columns = 'debt', values = 'dob_years', aggfunc='count')
 
#переименовываем колонки
new_columns = ['платежи в срок','просроченные платежи']
family_status_data.set_axis(new_columns, axis='columns',inplace = True)
 
#создаем новый столбец с долей и выводим таблицу на печать
family_status_data['доля просроченных платежей']=family_status_data['просроченные платежи']/(family_status_data['просроченные платежи']+family_status_data['платежи в срок'])
print(family_status_data)

                       платежи в срок  просроченные платежи  \
family_status                                                 
в разводе                        1110                    85   
вдовец / вдова                    896                    63   
гражданский брак                 3763                   388   
женат / замужем                 11408                   931   
не женат / не замужем            2536                   274   

                       доля просроченных платежей  
family_status                                      
в разводе                                0.071130  
вдовец / вдова                           0.065693  
гражданский брак                         0.093471  
женат / замужем                          0.075452  
не женат / не замужем                    0.097509  


### Вывод

Исходя из полученных данных можно обозначить следующее:
1. Самые ответственные плательщики - вдовы и вдовцы, а после них идут разведённые
2. Больше всех просрочек по платежам у не женатых или не узаконивших свои отношения (гражданский брак).
3. Женатые почти на четверть реже задерживают платежи, чем НЕ женатые.

Как итог, зависимость между семейным положением и возвратом кредита в срок есть

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.
    
</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
#создаем сводную таблицу
total_income_data=client_solvency.pivot_table(index = ['income_categories'], columns = 'debt', values = 'total_income', aggfunc='count')
 
#переименовываем колонки
new_columns = ['платежи в срок','просроченные платежи']
total_income_data.set_axis(new_columns, axis='columns',inplace = True)
 
#создаем новый столбец с долей и выводим таблицу на печать
total_income_data['доля просроченных платежей']=total_income_data['просроченные платежи']/(total_income_data['просроченные платежи']+total_income_data['платежи в срок'])
print(total_income_data)

                     платежи в срок  просроченные платежи  \
income_categories                                           
высокий доход                  5447                   447   
доход выше среднего            7475                   712   
низкий доход                    757                    49   
очень высокий доход             474                    29   
рублёвые миллионеры              23                     2   
средний доход                  5537                   502   

                     доля просроченных платежей  
income_categories                                
высокий доход                          0.075840  
доход выше среднего                    0.086967  
низкий доход                           0.060794  
очень высокий доход                    0.057654  
рублёвые миллионеры                    0.080000  
средний доход                          0.083126  


### Вывод

Исходя из полученных данных можно обозначить следующее:
1. Больше всех просрочки по платежам допускают люди с доходом менее 60 т.р. (низкий доход) и как ни странна люди с доходом от 400т.р. до 1 млн рублей.
2. Люди со средним доходом и выше среднего реже других задерживают платежи.
3. Миллионеры задерживают платежи чаще, чем люди со средним заработком.

Как итог, зависимость между уровнем дохода и возвратом кредита в срок есть. Лучше всех выполняет свои обязательства средний класс!

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Исходя из расчетов верно, но не маловажно понимать каким образом были сформированы данные категории доходов.
    
</div>

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
#создаем сводную таблицу
purpose_category_data=client_solvency.pivot_table(index = ['purpose_lemm_category'], columns = 'debt', values = ['purpose_index'], aggfunc='count')
 
#переименовываем колонки
new_columns = ['платежи в срок','просроченные платежи']
purpose_category_data.set_axis(new_columns, axis='columns',inplace = True)
 
#создаем новый столбец с долей и выводим таблицу на печать
purpose_category_data['доля просроченных платежей']=purpose_category_data['просроченные платежи']/(purpose_category_data['просроченные платежи']+purpose_category_data['платежи в срок'])
print(purpose_category_data)

                           платежи в срок  просроченные платежи  \
purpose_lemm_category                                             
Деньги на свадьбу                    2138                   186   
Коммерческая недвижимость            1212                    99   
Кредит на образование                3643                   370   
Покупка авто                         3903                   403   
Покупка недвижимости                 8817                   683   

                           доля просроченных платежей  
purpose_lemm_category                                  
Деньги на свадьбу                            0.080034  
Коммерческая недвижимость                    0.075515  
Кредит на образование                        0.092200  
Покупка авто                                 0.093590  
Покупка недвижимости                         0.071895  


### Вывод

Исходя из полученных данных можно обозначить следующее:
1. Платежи по ипотекам (кредитам на недвижимость) задерживают реже других кредитов.
2. Больше всего просрочек делают по кредитам на авто и образование
3. Просрочки платежей по кредитам на свадьбу совершают чаще, чем по кредитам на покупку недвижимости, но реже чем на образование и авто.

Вывод - ипотечные кредиты самые надёжные для банка при прочих равных.

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Верно.
    
</div>

### Шаг 4. Общий вывод

Многие факторы влияют на то, делают ли люди просрочки или нет. Попробуем составить портреты "идеального" заёмщика и "проблемного"

"Идеальный заёмщик"
- Вдовец или разведённый
- Без детей
- Берёт кредит на недвижимость (ипотеку)
- Доход от 120 до 400 т.р.

"Проблемный заёмщик":
- Не женат
- Количество детей - двое (откуда он их взял?)
- Берёт кредит на автомобиль
- Доход до 60 т.р. или от 400 т.р. до 1млн.


Я уверен, что для составления кредитного скоринга необходимо будет учесть и другие параметры, которые есть в предоставленных данных, но даже то, что есть уже показывает некоторые закономерности, по которым можно присваивать рейтинг платёжеспособности (кредитный скоринг) и использовать как инструмент для автоматизации одобрения или отказа в предоставлении кредита.



<div class="alert alert-warning">
<h2> Комментарий ревьюера</h2>

Финальный вывод и есть главный результат твоей работы. Стоит писать его подробно по результатам проведенной работы. В нем можно приводить полученные в ходе работы значения. Также можно расписать все, что было сделано в работе.
    
Сейчас финальный вывод отвечает на главный вопрос, и максимально логичен, а самое главное подкреплен фактами на основании нашего исследования.
    
</div>

<div class="alert alert-success">
<h2> Комментарий ревьюера</h2>

Работа выполнена на хорошем уровне! Использовано большое количество изученных методов, которые будут помогать тебе в будущих проектах. На протяжении всей работы чувствуется вся глубина проработки, это безусловный плюс. 
Рекомендую не стесняться использовать ячейки типа «markdown», в тех местах где нужно пояснить свои мысли и дополнительно стилизовать свои размышления. Оформление проекта — это также один из важнейших блоков всей работы.

Успехов тебе в будущих проектах, не сбавляй темпы своего обучения! :)
    
</div>

# Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.